In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
exe = pd.read_excel("../Historical Cable Reel Returns.xlsx")

In [4]:
exe.head()

,\nBill To,Bill To Entity,\nShip To,Ship To Entity,\nName,\nReel Serial #,\nReel Size,\nShip Date,\nReturn Date,Days\nOutstanding,...,\nDimension,\nCredit Memo,\nOriginal Invoice,Original\nShipping Invoice,Ship To\nPO,\nDeposit $,\nCredit Date,Amortization\nCharged,Damages\nCharged,\nCredit Amount
0,Y10019,Meade Electric,Y10019,Meade Electric,EXELON BUSINESS SERVICES,BC145024MOK,MOK,2015-04-09,2016-01-06,272,...,90x45x42,1526427.0,3359173,3319713.0,01149707,1550,2016-01-22,0,0.0,1550.0
1,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107127LOK,LOK,2016-06-23,2016-11-18,148,...,84x45x42,1541996.0,3713223,3672770.0,01149707,1400,2016-12-07,0,0.0,1400.0
2,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107555LOK,LOK,2016-04-14,2017-03-15,335,...,84x45x42,1546991.0,3713220,3615793.0,01149707,1400,2017-03-24,0,0.0,1400.0
3,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107620LOK,LOK,2016-04-13,2017-04-27,379,...,84x45x42,1549369.0,3713218,3615791.0,01149707,1400,2017-05-12,0,280.0,1120.0
4,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107871LOK,LOK,2016-04-14,2016-11-17,217,...,84x45x42,1541991.0,3713220,3615794.0,01149707,1400,2016-12-07,0,0.0,1400.0


In [5]:
header_row = exe.columns
header_row = [header.replace('\n', '') for header in header_row]
exe.columns = header_row

In [6]:
exe.shape

(7108, 24)

In [7]:
exe.columns

Index(['Bill To', 'Bill To Entity', 'Ship To', 'Ship To Entity', 'Name',
       'Reel Serial #', 'Reel Size', 'Ship Date', 'Return Date',
       'DaysOutstanding', 'Order #', 'OriginalPart', 'Include in Analysis',
       'MPG', 'Dimension', 'Credit Memo', 'Original Invoice',
       'OriginalShipping Invoice', 'Ship ToPO', 'Deposit $', 'Credit Date',
       'AmortizationCharged', 'DamagesCharged', 'Credit Amount'],
      dtype='object')

In [8]:
exe = exe.rename(columns={'OriginalPart': 'Original Part', 
                            'OriginalShipping Invoice': 'Original Shipping Invoice', 
                            'Ship ToPO': 'Ship To PO',
                            'Deposit $': 'Deposit', 
                            'AmortizationCharged': 'Amortization Charged',
                            'DaysOutstanding': 'Days Outstanding', 
                            'DamagesCharged': 'Damages Charged'
                            })

In [9]:
exe.dtypes

Bill To                              object
Bill To Entity                       object
Ship To                              object
Ship To Entity                       object
Name                                 object
Reel Serial #                        object
Reel Size                            object
Ship Date                    datetime64[ns]
Return Date                  datetime64[ns]
Days Outstanding                      int64
Order #                               int64
Original Part                        object
Include in Analysis                  object
MPG                                  object
Dimension                            object
Credit Memo                         float64
Original Invoice                      int64
Original Shipping Invoice           float64
Ship To PO                           object
Deposit                               int64
Credit Date                  datetime64[ns]
Amortization Charged                  int64
Damages Charged                 

In [10]:
#columns_to_drop = ['Bill To', 'Ship To', 'Name', 'Reel Size', 'Order #', 'Original Part', 'MPG', 'Original Invoice', 'Original Shipping Invoice', 'Ship To PO']
#exe = exe.drop(columns = columns_to_drop)

In [11]:
exe.head()

,Bill To,Bill To Entity,Ship To,Ship To Entity,Name,Reel Serial #,Reel Size,Ship Date,Return Date,Days Outstanding,...,Dimension,Credit Memo,Original Invoice,Original Shipping Invoice,Ship To PO,Deposit,Credit Date,Amortization Charged,Damages Charged,Credit Amount
0,Y10019,Meade Electric,Y10019,Meade Electric,EXELON BUSINESS SERVICES,BC145024MOK,MOK,2015-04-09,2016-01-06,272,...,90x45x42,1526427.0,3359173,3319713.0,01149707,1550,2016-01-22,0,0.0,1550.0
1,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107127LOK,LOK,2016-06-23,2016-11-18,148,...,84x45x42,1541996.0,3713223,3672770.0,01149707,1400,2016-12-07,0,0.0,1400.0
2,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107555LOK,LOK,2016-04-14,2017-03-15,335,...,84x45x42,1546991.0,3713220,3615793.0,01149707,1400,2017-03-24,0,0.0,1400.0
3,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107620LOK,LOK,2016-04-13,2017-04-27,379,...,84x45x42,1549369.0,3713218,3615791.0,01149707,1400,2017-05-12,0,280.0,1120.0
4,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107871LOK,LOK,2016-04-14,2016-11-17,217,...,84x45x42,1541991.0,3713220,3615794.0,01149707,1400,2016-12-07,0,0.0,1400.0


In [12]:
new_column_names = {'\nReel Serial #': 'Reel Serial #', '\nShip Date': 'Ship Date', '\nReturn Date': 'Return Date', 'Days\nOutstanding': 'Days Outstanding', '\nDimension' : 'Dimension', '\nCredit Memo': 'Credit Memo', '\nDeposit $': 'Deposit', '\nCredit Date' : 'Credit Date', 'Amortization\nCharged': 'Amortization Charged', 'Damages\nCharged': 'Damages Charged', '\nCredit Amount': 'Credit Amount'}

In [13]:
exe = exe.rename(columns=new_column_names)

In [14]:
exe.head()

,Bill To,Bill To Entity,Ship To,Ship To Entity,Name,Reel Serial #,Reel Size,Ship Date,Return Date,Days Outstanding,...,Dimension,Credit Memo,Original Invoice,Original Shipping Invoice,Ship To PO,Deposit,Credit Date,Amortization Charged,Damages Charged,Credit Amount
0,Y10019,Meade Electric,Y10019,Meade Electric,EXELON BUSINESS SERVICES,BC145024MOK,MOK,2015-04-09,2016-01-06,272,...,90x45x42,1526427.0,3359173,3319713.0,01149707,1550,2016-01-22,0,0.0,1550.0
1,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107127LOK,LOK,2016-06-23,2016-11-18,148,...,84x45x42,1541996.0,3713223,3672770.0,01149707,1400,2016-12-07,0,0.0,1400.0
2,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107555LOK,LOK,2016-04-14,2017-03-15,335,...,84x45x42,1546991.0,3713220,3615793.0,01149707,1400,2017-03-24,0,0.0,1400.0
3,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107620LOK,LOK,2016-04-13,2017-04-27,379,...,84x45x42,1549369.0,3713218,3615791.0,01149707,1400,2017-05-12,0,280.0,1120.0
4,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107871LOK,LOK,2016-04-14,2016-11-17,217,...,84x45x42,1541991.0,3713220,3615794.0,01149707,1400,2016-12-07,0,0.0,1400.0


In [15]:
exe.dtypes

Bill To                              object
Bill To Entity                       object
Ship To                              object
Ship To Entity                       object
Name                                 object
Reel Serial #                        object
Reel Size                            object
Ship Date                    datetime64[ns]
Return Date                  datetime64[ns]
Days Outstanding                      int64
Order #                               int64
Original Part                        object
Include in Analysis                  object
MPG                                  object
Dimension                            object
Credit Memo                         float64
Original Invoice                      int64
Original Shipping Invoice           float64
Ship To PO                           object
Deposit                               int64
Credit Date                  datetime64[ns]
Amortization Charged                  int64
Damages Charged                 

In [16]:
exe['Ship Date'] = pd.to_datetime(exe['Ship Date'], dayfirst=True)
exe['Return Date'] = pd.to_datetime(exe['Return Date'], dayfirst=True)
exe['Credit Date'] = pd.to_datetime(exe['Credit Date'], dayfirst=True)
exe['Include in Analysis'] = exe['Include in Analysis'].astype('category')

In [17]:
exe.dtypes

Bill To                              object
Bill To Entity                       object
Ship To                              object
Ship To Entity                       object
Name                                 object
Reel Serial #                        object
Reel Size                            object
Ship Date                    datetime64[ns]
Return Date                  datetime64[ns]
Days Outstanding                      int64
Order #                               int64
Original Part                        object
Include in Analysis                category
MPG                                  object
Dimension                            object
Credit Memo                         float64
Original Invoice                      int64
Original Shipping Invoice           float64
Ship To PO                           object
Deposit                               int64
Credit Date                  datetime64[ns]
Amortization Charged                  int64
Damages Charged                 

In [18]:
exe.shape

(7108, 24)

In [19]:
exe.nunique()

Bill To                         2
Bill To Entity                  2
Ship To                        12
Ship To Entity                  7
Name                            6
Reel Serial #                5615
Reel Size                      17
Ship Date                    1191
Return Date                   683
Days Outstanding             1519
Order #                      1228
Original Part                  50
Include in Analysis             1
MPG                             4
Dimension                      12
Credit Memo                  6721
Original Invoice             2706
Original Shipping Invoice    2875
Ship To PO                     36
Deposit                        25
Credit Date                   407
Amortization Charged            9
Damages Charged                68
Credit Amount                  87
dtype: int64

In [20]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [21]:
# Calculate the date difference and create a new column to store it
exe['Date Difference'] = exe['Return Date'] - exe['Ship Date']

In [22]:
exe['Deposit Lost'] = exe['Deposit'] - exe['Credit Amount']

In [23]:
exe['Deposit Lost'].round()

0         0.00
1         0.00
2         0.00
3       280.00
4         0.00
         ...  
7103      0.00
7104      0.00
7105      0.00
7106    560.00
7107   1400.00
Name: Deposit Lost, Length: 7108, dtype: float64

In [24]:
exe['Deposit Lost'].astype(int)

0          0
1          0
2          0
3        280
4          0
        ... 
7103       0
7104       0
7105       0
7106     560
7107    1400
Name: Deposit Lost, Length: 7108, dtype: int32

In [25]:
exe.head()

,Bill To,Bill To Entity,Ship To,Ship To Entity,Name,Reel Serial #,Reel Size,Ship Date,Return Date,Days Outstanding,...,Original Invoice,Original Shipping Invoice,Ship To PO,Deposit,Credit Date,Amortization Charged,Damages Charged,Credit Amount,Date Difference,Deposit Lost
0,Y10019,Meade Electric,Y10019,Meade Electric,EXELON BUSINESS SERVICES,BC145024MOK,MOK,2015-04-09,2016-01-06,272,...,3359173,3319713.00,01149707,1550,2016-01-22,0,0.00,1550.00,272 days,0.00
1,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107127LOK,LOK,2016-06-23,2016-11-18,148,...,3713223,3672770.00,01149707,1400,2016-12-07,0,0.00,1400.00,148 days,0.00
2,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107555LOK,LOK,2016-04-14,2017-03-15,335,...,3713220,3615793.00,01149707,1400,2017-03-24,0,0.00,1400.00,335 days,0.00
3,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107620LOK,LOK,2016-04-13,2017-04-27,379,...,3713218,3615791.00,01149707,1400,2017-05-12,0,280.00,1120.00,379 days,280.00
4,Y10019,Meade Electric,002062,Meade Electric,EXELON BUSINESS SERVICES,BC107871LOK,LOK,2016-04-14,2016-11-17,217,...,3713220,3615794.00,01149707,1400,2016-12-07,0,0.00,1400.00,217 days,0.00


In [26]:
(exe['Deposit Lost'] == 0).sum()

3818

In [27]:
exe['Deposit Lost'].describe()

count   7108.00
mean     309.04
std      481.18
min        0.00
25%        0.00
50%        0.00
75%      310.00
max     3000.00
Name: Deposit Lost, dtype: float64

In [28]:
#exe['Deposit Lost'] = np.where(exe['Deposit Lost'] == 0, pd.NA, exe['Deposit Lost'])

# All columns created

In [29]:
exe.shape

(7108, 26)

In [30]:
exe.describe()

,Ship Date,Return Date,Days Outstanding,Order #,Credit Memo,Original Invoice,Original Shipping Invoice,Deposit,Credit Date,Amortization Charged,Damages Charged,Credit Amount,Date Difference,Deposit Lost
count,7108,7108,7108.00,7108.00,7095.00,7108.00,7104.00,7108.00,7095,7108.00,7108.00,7108.00,7108,7108.00
mean,2018-03-12 03:12:51.862690048,2019-05-25 14:56:27.281935872,439.49,558900.15,1578965.61,4310605.87,4308445.22,1479.04,2019-06-03 17:47:46.300211200,151.37,157.67,1170.00,439 days 11:43:35.419245920,309.04
min,2009-06-11 00:00:00,2016-01-06 00:00:00,5.00,1857.00,1526138.00,3000551.00,3000551.00,0.00,2016-01-18 00:00:00,0.00,0.00,0.00,5 days 00:00:00,0.00
25%,2016-08-11 00:00:00,2017-06-13 00:00:00,125.00,305081.00,1551002.50,3758741.00,3758741.75,1400.00,2017-06-21 00:00:00,0.00,0.00,1120.00,125 days 00:00:00,0.00
50%,2017-12-11 00:00:00,2019-04-03 00:00:00,247.00,602126.00,1577678.00,4119045.00,4119045.00,1400.00,2019-03-22 00:00:00,0.00,0.00,1400.00,247 days 00:00:00,0.00
75%,2020-02-04 00:00:00,2021-04-14 00:00:00,527.00,808663.00,1607455.50,4671509.00,4671509.00,1550.00,2021-04-28 00:00:00,0.00,280.00,1400.00,527 days 00:00:00,310.00
max,2023-10-24 00:00:00,2023-12-19 00:00:00,3692.00,999549.00,1635668.00,8873346.00,8873346.00,13000.00,2024-01-02 00:00:00,3000.00,2240.00,13000.00,3692 days 00:00:00,3000.00
std,NaN,NaN,514.92,285144.22,30785.64,947327.71,949076.05,315.88,NaN,447.16,281.65,561.46,514 days 22:04:57.472056912,481.18


In [31]:
def get_data():
    return exe